Есть две коллекции (таблицы) данных: accrual (долги) и payment (платежи). Обе коллекции имеют поля:
- id
- date (дата)
- month (месяц)

Необходимо написать функцию, которая сделает запрос к платежам и найдёт для каждого платежа долг, который будет им оплачен. Платёж может оплатить только долг, имеющий более раннюю дату. Один платёж может оплатить только один долг, и каждый долг может быть оплачен только одним платежом. Платёж приоритетно должен выбрать долг с совпадающим месяцем (поле month). Если такого нет, то самый старый по дате (поле date) долг.
Результатом должна быть таблица найденных соответствий, а также список платежей, которые не нашли себе долг.
Запрос можно делать к любой базе данных (mongodb, postgresql или другие) любым способом.

In [5]:
import datetime
import pandas as pd

Для решения этой задачи мной была выбрана следующая структура входных данных:

In [6]:
accruals = [
    {'id': '01', 'date': datetime.datetime(2019, 1, 20), 'month': 1},
    {'id': '02', 'date': datetime.datetime(2019, 2, 20), 'month': 2},
    {'id': '03', 'date': datetime.datetime(2019, 3, 20), 'month': 3},
    {'id': '04', 'date': datetime.datetime(2019, 4, 20), 'month': 4},
    {'id': '05', 'date': datetime.datetime(2019, 5, 20), 'month': 5},
    {'id': '06', 'date': datetime.datetime(2019, 6, 20), 'month': 6},
    {'id': '07', 'date': datetime.datetime(2019, 7, 20), 'month': 7},
    {'id': '08', 'date': datetime.datetime(2019, 8, 20), 'month': 8},
    {'id': '09', 'date': datetime.datetime(2019, 9, 20), 'month': 9},
    {'id': '10', 'date': datetime.datetime(2019, 10, 20), 'month': 10},
    {'id': '11', 'date': datetime.datetime(2019, 11, 20), 'month': 11},
    {'id': '12', 'date': datetime.datetime(2019, 12, 20), 'month': 12},
]

payments = [
    {'id': '01', 'date': datetime.datetime(2019, 1, 25), 'month': 1},
    {'id': '02', 'date': datetime.datetime(2019, 4, 22), 'month': 4},
    {'id': '03', 'date': datetime.datetime(2019, 5, 23), 'month': 5},
    {'id': '04', 'date': datetime.datetime(2019, 5, 24), 'month': 5},
    {'id': '05', 'date': datetime.datetime(2019, 5, 25), 'month': 5},
    {'id': '06', 'date': datetime.datetime(2019, 5, 21), 'month': 5},
    {'id': '07', 'date': datetime.datetime(2019, 8, 22), 'month': 8},
    {'id': '08', 'date': datetime.datetime(2019, 8, 25), 'month': 8},
    {'id': '09', 'date': datetime.datetime(2019, 8, 26), 'month': 8},
    {'id': '10', 'date': datetime.datetime(2019, 10, 22), 'month': 10},
    {'id': '11', 'date': datetime.datetime(2019, 11, 22), 'month': 11},
    {'id': '12', 'date': datetime.datetime(2019, 12, 25), 'month': 12},
    {'id': '13', 'date': datetime.datetime(2019, 12, 26), 'month': 12},
]

matches_pay = []
matches_debt = []
unmatches = []
offset = 0

Функция control() обрабатывает 2 входящих списка в соответствии с заданным отступом. Поскольку мы знаем, что данные поступают массив 'accruals' в хронологическом порядке, после успешного закрытия n-го долга (т.е, нахождения соответствия) мы можем исключить данный n-й элемент 'acc_itm' из анализируемой выборки путем увеличения отступа 'offset, а соответственно и сужением выборки при каждом успешном проходе цикла. 

При совпадении с заданным условием, все последующие элементы в 'accruals' для данного платежа 'pay_itm' можно проигнорировать прерыванием цикла.

In [7]:
def control(payments, accruals, offset):
    for pay_itm in payments:
        for acc_itm in accruals[offset:]:
            if pay_itm['date'] > acc_itm['date']:
                matches_pay.append(pay_itm['id'])
                matches_debt.append(acc_itm['id'])
                offset += 1
                break
            elif pay_itm['id'] not in unmatches:
                unmatches.append(pay_itm['id'])

    return {'debt_id': matches_debt, 'payment_id': matches_pay}

In [8]:
%%time
if __name__ == '__main__':
    result = control(payments, accruals, offset)
    print(pd.DataFrame(result))
    print(f'Didn\'t find a debt to pay for: {unmatches}\n')

   debt_id payment_id
0       01         01
1       02         02
2       03         03
3       04         04
4       05         05
5       06         07
6       07         08
7       08         09
8       09         10
9       10         11
10      11         12
11      12         13
Didn't find a debt to pay for: ['06']

CPU times: user 3.9 ms, sys: 106 µs, total: 4 ms
Wall time: 5.07 ms
